# CLIP data processing pipeline:
- [ ] Convert textual and visual data into a dict
- [ ] Create train-test-dev splits
- [ ] Train model on train data
- [ ] Test the model


In [32]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys

sys.path.append('/Users/mhendriksen/Desktop/repositories/CLIP/src')

In [17]:
import os

root = '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/'

DATASET = 'CUB'

files = os.listdir(root)

files_filtered = [file for file in files if DATASET in file]

files_filtered

['CUB_csv_prep.ipynb',
 'best_model_CUB_30E_128BS.pt',
 'best_model_CUB_30E_128BS_image_to_text.pkl',
 'CUB_200_2011']

In [4]:
from utils import load_pkl

path = '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/best_model_CUB_30E_128BS_image_to_text.pkl'

df = load_pkl(path)

Loaded data from  /Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/best_model_CUB_30E_128BS_image_to_text.pkl


In [9]:
df.describe()

,i2t_query_ids,i2t_recalls_at_1,i2t_recalls_at_5,i2t_recalls_at_10,i2t_recalls_at_50,i2t_recalls_at_100
count,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000
mean,58509.341389,0.001021,0.003063,0.006127,0.024506,0.052076
std,33579.145458,0.031944,0.055272,0.078046,0.154642,0.222219
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29308.500000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,58722.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,87608.250000,0.000000,0.000000,0.000000,0.000000,0.000000
max,116554.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:
def recall_at_k(target, predictions, k=5, total_in_collection=1):
    if len(predictions) < k:
        print(f'Watch out, len(predictions) < k: {len(predictions)} < {k}')
    recall_at_k = predictions[:k].count(target) / total_in_collection
    return 100 * recall_at_k


t = 5
preds = [1,3,5]

recall_at_k(t, preds)

Watch out, len(predictions) < k: 3 < 5


100.0

In [98]:
pd.read_csv('/Users/mhendriksen/Desktop/repositories/datasets/flickr30k_images/dataset_flickr30k.csv', index_col=0)

,eval_status,image_caption,image_name
0,train,Two young guys with shaggy hair look at their ...,1000092795.jpg
5,train,Several men in hard hats are operating a giant...,10002456.jpg
10,train,A child in a pink dress is climbing up a set o...,1000268201.jpg
15,train,Someone in a blue shirt and hat is standing on...,1000344755.jpg
20,train,"Two men, one in a gray shirt, one in a black s...",1000366164.jpg
...,...,...,...
155045,train,"Woman writing on a pad in room with gold, deco...",997338199.jpg
155050,train,A person in a red shirt climbing up a rock fac...,997722733.jpg
155055,train,Two male construction workers are working on a...,997876722.jpg
155060,train,An older busker in glasses plays an Eastern st...,99804383.jpg


In [130]:
df = pd.read_csv('/Users/mhendriksen/Desktop/repositories/datasets/fashion-200k/fashion200k.csv', index_col=0)

In [136]:
len(df.index.tolist())

201838

In [137]:
len(df.index.unique().tolist())

172049

In [3]:
# ################################################
# # attempt to reproduce clip loss as in the paper
# ################################################
# temperature = 1
# t = temperature
# n = batch_size = 3

# # extract feature representations of each modality
# I_f = torch.rand(n, 768)
# T_f = torch.rand(n, 768)


# # joint multimodal embedding
# I_e = l2_normalize(projection.forward(I_f))
# T_e = l2_normalize(projection.forward(T_f))


# # scaled pairwise cosine similarities [n, n]
# I_e = I_e.detach().numpy()
# T_e = T_e.detach().numpy()
# logits = np.dot(I_e, T_e.T) * np.exp(t)


# # import numpy as np
# # # symmetric loss functio
# # labels = np.arange(n)
# # loss_image = cross_entropy(logits, labels)

In [88]:
import yaml
import torch

############################
# load config files
############################
train_config_file = '/Users/mhendriksen/Desktop/repositories/CLIP/conf/train_conf.yaml'
dataset_config_file = '/Users/mhendriksen/Desktop/repositories/CLIP/conf/local_cub_conf.yaml'

with open(dataset_config_file) as file:
    config_ds = yaml.safe_load(file)
print(config_ds)

with open(train_config_file) as file:
    config_train = yaml.safe_load(file)
print(config_train)

config_train["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

{'dataset': 'CUB_200', 'dataset_root': '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/', 'csv_file': 'cub_1_cap_per_img.csv', 'image_folder': 'CUB_200_2011/images', 'columns_dtypes': {'image_file': 'str', 'class': 'str', 'eval_status': 'str', 'caption': 'str'}, 'content_type': {'image': 'image_file', 'text': 'caption'}, 'clip_version': 'clip-ViT-L-14', 'dict_file': 'test_dict.pkl'}
{'DATASET': {'CUB': {'text_dict_path': '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/text_dict.pkl', 'image_dict_path': '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/image_dict.pkl', 'df_train_path': '../data/CUB/df_train.csv', 'df_test_path': '../data/CUB/df_test.csv'}}, 'clip_version': 'clip-ViT-L-14', 'deliverables_folder': '../deliverables', 'batch_size': 128, 'epochs': 30, 'device': 'to be defined in the script', 'head_lr': 0.001, 'weight_decay': 0.001, 'patience': 1, 'factor': 0.8, 'text_input_dim': 768, 'text_hidden_dim': 2048, 'image_input_dim': 768, 

In [90]:
config_ds['CUB']

KeyError: 'CUB'

In [5]:
# import pandas as pd
# import os

# # read df
# df = pd.read_csv(
#     filepath_or_buffer=os.path.join(config_ds['dataset_root'], config_ds['csv_file']),
#     dtype=config_ds['columns_dtypes'],
#     index_col=0

#     )
# df.head()

# from utils import make_train_test_dev_dfs
# # build train and valid loaders
# df_train, df_test, df_dev = make_train_test_dev_dfs(df)
# TODO: save the dfs

List of released CLIP models as of 18 Aug 2022:

| Model         | Top 1 Performance |
| :------------ | :---------------- |
| clip-ViT-B-32 | 63.3              |
| clip-ViT-B-16 | 68.1              |
| clip-ViT-L-14 | 75.4              |

In [1]:
from sentence_transformers import SentenceTransformer
clip_model = SentenceTransformer('clip-ViT-L-14')

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [6]:
text_dict_path = '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/text_dict.pkl'
image_dict_path = '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/image_dict.pkl'

from utils import load_pkl

image_dict = load_pkl(image_dict_path)
text_dict = load_pkl(text_dict_path)

Loaded data from  /Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/image_dict.pkl
Loaded data from  /Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/text_dict.pkl


In [7]:
import pandas as pd

df_train = pd.read_csv('df_train.csv', index_col=0)
df_test = pd.read_csv('df_test.csv', index_col=0)

In [8]:
from dataset import ImageCaptionDataset, build_loader

train_loader = build_loader(
    dataset=ImageCaptionDataset,
    config=config_train,
    dataf=df_train,
    text_dict=text_dict,
    image_dict=image_dict,
    mode='train'
    )
valid_loader = build_loader(
    dataset=ImageCaptionDataset,
    config=config_train,
    dataf=df_test,
    text_dict=text_dict,
    image_dict=image_dict,
    mode='test'
    )


In [9]:
from model import CLIPModel
import itertools
# from util import cross_entropy

model = CLIPModel(config=config_train).to(config_train["device"])

# get optimizer
params = [
    {
        'params': itertools.chain(
            model.image_projection_head.parameters(),
            model.text_projection_head.parameters()
        ),
        'lr': config_train['head_lr'],
        'weight_decay': config_train['weight_decay']
    }
]
optimizer = torch.optim.AdamW(params)

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    patience=config_train["patience"],
    factor=config_train["factor"]
)


In [103]:
lr_scheduler.step

NameError: name 'lr_scheduler' is not defined

In [41]:
from train_valid import train_epoch, valid_epoch

step = "epoch"
train_loss_values = []
valid_loss_values = []
best_loss = float('inf')
for epoch in range(config_train["epochs"]):
    running_loss = 0.0
    print(f"Epoch: {epoch + 1}")
    model.train()
    train_loss = train_epoch(config_train, model, train_loader, optimizer, lr_scheduler, step)
    # print(train_loss.avg)
    train_loss_values.append(train_loss.avg)
    
    model.eval()
    with torch.no_grad():
        valid_loss = valid_epoch(config_train, model, valid_loader)
        valid_loss_values.append(valid_loss.avg)

    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg

        # dt_string = get_dt_string()
        torch.save(model.state_dict(), "test_model.pt")
        print(f"Saved Best Model! best loss - {best_loss}")
        # torch.save(valid_loader, f"{CFG.deliverables_path}/{dt_string}_valid_loader.pth")
        # print('Saved the dataloader!')

    lr_scheduler.step(valid_loss.avg)

Epoch: 1


100%|██████████| 188/188 [00:28<00:00,  6.60it/s, lr=0.001, train_loss=3.31]


3.3076368127618587


100%|██████████| 90/90 [00:02<00:00, 43.60it/s, valid_loss=3.45]


Saved Best Model! best loss - 3.4483796575156247
Epoch: 2


100%|██████████| 188/188 [00:28<00:00,  6.58it/s, lr=0.0008, train_loss=3.3]


3.2985007851212114


100%|██████████| 90/90 [00:02<00:00, 43.21it/s, valid_loss=3.44]


Saved Best Model! best loss - 3.4434459783783815


In [148]:
# evaluation pipeline
from sentence_transformers import util

In [119]:
import pickle, yaml
import torch

def load_pkl(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
        print('Loaded data from ', path)
    return data

text_dict_path = '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/text_dict.pkl'
image_dict_path = '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/image_dict.pkl'

image_dict = load_pkl(image_dict_path)
text_dict = load_pkl(text_dict_path)

dataset_config_file = '/Users/mhendriksen/Desktop/repositories/CLIP/conf/local_cub_conf.yaml'
with open(dataset_config_file) as file:
    config_ds = yaml.safe_load(file)
print(config_ds)

eval_config_file = '../conf/train_conf.yaml'
with open(eval_config_file) as file:
    config = yaml.safe_load(file)

config["device"] = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Loaded data from  /Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/image_dict.pkl
Loaded data from  /Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/text_dict.pkl
{'CUB': {'dataset_root': '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/', 'csv_file': 'cub_1_cap_per_img.csv', 'image_folder': 'CUB_200_2011/images', 'columns_dtypes': {'image_file': 'str', 'class': 'str', 'eval_status': 'str', 'caption': 'str'}, 'content_type': {'image': 'image_file', 'text': 'caption'}, 'clip_version': 'clip-ViT-L-14', 'dict_file': 'test_dict.pkl'}}


In [120]:
path = config['DATASET']['CUB']['model_path']

os.path.exists(path)

True

In [114]:
model_path

'/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/best_model_CUB_30E_128BS.pt'

In [5]:
from model import CLIPModel


print('Loading pre-trained model...')
model_path = '/Users/mhendriksen/Desktop/repositories/datasets/CUB_200_2011/best_model_CUB_30E_128BS.pt'

def get_model(path, config):
    model = CLIPModel(config)
    model.load_state_dict(torch.load(path, map_location=config["device"]))
    model.eval()
    return model

model = get_model(path=model_path, config=config)

Loading pre-trained model...


In [9]:
# get queries
# get image

dataset = 'CUB'

# load dev df

print('Load dev df...')
df_dev_path = '/Users/mhendriksen/Desktop/repositories/CLIP/data/CUB/df_dev.csv'

import pandas as pd

df_dev = pd.read_csv(df_dev_path, index_col=0)

print('Get ids, images and captions from dev df... ')
ids = df_dev.index.tolist()
images = df_dev[config_ds[dataset]['content_type']['image']].tolist()
captions = df_dev[config_ds[dataset]['content_type']['text']].tolist()

assert len(images) == len(captions)

Load dev df...
Get ids, images and captions from dev df... 


In [102]:
from evaluation_utils import dict_to_tensor, dict_to_list, get_top_k_indices
from tqdm.notebook import tqdm


print('Text to image retrieval results...')
print('Build a list of textual query embeddings and an images tensor...')
text_query_embs = dict_to_list(
    indices=ids,
    dict=text_dict,
    trained_model=model,
    content_type='text'
)
images_tensor = dict_to_tensor(
    indices=ids,
    dict=image_dict,
    trained_model=model,
    content_type='image',
    dim=0,
    clip_version='clip-ViT-L-14'
)
assert len(text_query_embs) == len(images_tensor)

Text to image retrieval results...
Build a list of textual query embeddings and an images tensor...


100%|██████████| 2938/2938 [00:05<00:00, 574.01it/s]


Got a list of 2938 embeddings


100%|██████████| 2938/2938 [00:04<00:00, 612.93it/s]

Final tensor shape: torch.Size([2938, 2048])


In [129]:
os.path.basename(os.path.splitext(model_path)[0])

'best_model_CUB_30E_128BS'

In [99]:
torch.squeeze(images_tensor, dim=0).shape

torch.Size([2938, 2048])

In [82]:
from evaluation_utils import numpy_to_tensor


t = torch.from_numpy(image_dict['data'][0]['clip-ViT-L-14'])
print(t.shape)
t = torch.unsqueeze(t,0)
print(t.shape)
t = model.image_projection_head(t)

l = [t, t]

torch.stack(l, dim=1).shape

torch.Size([768])
torch.Size([1, 768])


torch.Size([1, 2, 2048])

In [59]:
from evaluation_utils import numpy_to_tensor


test = torch.from_numpy(image_dict['data'][0]['clip-ViT-L-14'])

#test = torch.unsqueeze(test, 0)

tmp = [test, test]

numpy_to_tensor(tmp, dim=0).shape

torch.Size([2, 768])

In [94]:
from evaluation_utils import get_top_k_indices, recall_at_k, get_top_k_matches_for_queries

k=1000
print(f'For each text query, retrieve top k={k} results...')
text_to_image_results = get_top_k_matches_for_queries(
    indices=ids,
    queries_embs=text_query_embs,
    target_tensor=images_tensor,
    k=k,
    normalized=True
)
assert len(text_to_image_results) == len(ids) == len(images_tensor)

For each text query, retrieve top k=1000 results...


2938it [00:36, 79.75it/s]


In [95]:
from evaluation_utils import get_mrr, recall_at_k, get_recalls, iterate_over_results_to_compute_recalls

print('Calculating recall@k for text-to-image results...')
t2i_query_ids, t2i_top_k_results, t2i_recalls_at_1, t2i_recalls_at_5, t2i_recalls_at_10, t2i_recalls_at_50, t2i_recalls_at_100 = iterate_over_results_to_compute_recalls(
    ids=ids,
    results=text_to_image_results,
    k_list=[1, 5, 10, 50, 100]
)

t2i_results = pd.DataFrame(
    data={
       't2i_query_ids': t2i_query_ids,
        # text to image
        't2i_top_k_results': t2i_top_k_results,
        't2i_recalls_at_1': t2i_recalls_at_1,
        't2i_recalls_at_5': t2i_recalls_at_5,
        't2i_recalls_at_10': t2i_recalls_at_10,
        't2i_recalls_at_50': t2i_recalls_at_50,
        't2i_recalls_at_100': t2i_recalls_at_100,
    }
)

t2i_results.describe()

Calculating recall@k for text-to-image results...


2938it [00:00, 131507.02it/s]


,t2i_query_ids,t2i_recalls_at_1,t2i_recalls_at_5,t2i_recalls_at_10,t2i_recalls_at_50,t2i_recalls_at_100
count,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000
mean,58509.341389,0.000681,0.003063,0.005786,0.027910,0.061607
std,33579.145458,0.026086,0.055272,0.075860,0.164743,0.240480
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29308.500000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,58722.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,87608.250000,0.000000,0.000000,0.000000,0.000000,0.000000
max,116554.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [160]:
print('Image to text retrieval...')
print('Build a list of visual query embeddings and a captions tensor...')
image_query_embs = dict_to_list(
    indices=ids,
    dict=image_dict
)
text_tensor = dict_to_tensor(
    indices=ids,
    dict=text_dict,
    clip_version='clip-ViT-L-14'
)

Image to text retrieval results...
Build a list of visual query embeddings and a captions tensor...


100%|██████████| 2938/2938 [00:00<00:00, 300323.29it/s]


Got a list of 2938 embeddings


100%|██████████| 2938/2938 [00:00<00:00, 316384.64it/s]

Final tensor shape: torch.Size([2938, 768])


In [162]:
from evaluation_utils import get_top_k_indices, recall_at_k, get_top_k_matches_for_queries

k=1000
print(f'For each image query, retrieve top k={k} captions...')
image_to_text_results = get_top_k_matches_for_queries(
    indices=ids,
    queries_embs=image_query_embs,
    target_tensor=text_tensor,
    k=k,
    normalized=True
)
assert len(image_to_text_results) == len(ids) == len(text_tensor)

For each image query, retrieve top k=1000 captions...


2938it [00:20, 140.38it/s]


In [163]:
from evaluation_utils import get_mrr, recall_at_k, get_recalls, iterate_over_results_to_compute_recalls

print('Calculating recall@k for image-to-text results...')
i2t_query_ids, i2t_top_k_results, i2t_recalls_at_1, i2t_recalls_at_5, i2t_recalls_at_10, i2t_recalls_at_50, i2t_recalls_at_100 = iterate_over_results_to_compute_recalls(
    ids=ids,
    results=image_to_text_results,
    k_list=[1, 5, 10, 50, 100]
)

i2t_results = pd.DataFrame(
    data={
       'i2t_query_ids': i2t_query_ids,
        # text to image
        'i2t_top_k_results': i2t_top_k_results,
        'i2t_recalls_at_1': i2t_recalls_at_1,
        'i2t_recalls_at_5': i2t_recalls_at_5,
        'i2t_recalls_at_10': i2t_recalls_at_10,
        'i2t_recalls_at_50': i2t_recalls_at_50,
        'i2t_recalls_at_100': i2t_recalls_at_100,
    }
)

i2t_results.describe()

Calculating recall@k for image-to-text results...


2938it [00:00, 109332.49it/s]


,i2t_query_ids,i2t_recalls_at_1,i2t_recalls_at_5,i2t_recalls_at_10,i2t_recalls_at_50,i2t_recalls_at_100
count,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000
mean,58509.341389,0.012253,0.034037,0.055140,0.165759,0.256978
std,33579.145458,0.110033,0.181354,0.228291,0.371927,0.437041
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,29308.500000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,58722.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,87608.250000,0.000000,0.000000,0.000000,0.000000,1.000000
max,116554.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [104]:
from evaluation_utils import recall_at_k, get_mrr
from tqdm.notebook import tqdm

# iterate over the retrieved results,
# compute mrr, recall@k where k = {1, 5, 10, 50, 100}
k = 100
for query_id, result in tqdm(zip(query_ids, retrieved_results)):

    recall_at_100 = recall_at_k(
        target=query_id,
        predictions=result,
        k=k,
        total_in_collection=1
    )
    mrr = get_mrr(
        target=query_id,
        predictions=result
    )
    if recall_at_100 > 0.0:
        print(recall_at_100)

0it [00:00, ?it/s]

1.0
1.0


mrr

In [38]:

class RecallMeter:
    def __init__(self, target, predictions, total_in_collection=1, name='T2i| i2t recall'):
        self.target = target
        self.predictions = predictions
        self.total_in_collection = total_in_collection
        self.name = name

    def recall_at_k(self, k=5):
        self.predictions[:k].count(self.target) / self.total_in_collection

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text


t2i_recall = RecallMeter()

2

In [47]:
import torch 
import torch.nn.functional as F

normalized = True
k=5


image_index = numpy_to_tensor(image_index)
matches, values = get_top_k_indices(
    query_emb=query_emb,
    index=ids,
    target_embeddings=image_index,
    k=k,
    normalized=True
    )

predictions = df_dev.loc[matches]['image_file'].tolist()

matches

[53619, 53699, 2160, 1866, 10006]

In [33]:
id in matches

False

In [34]:
id

0

In [28]:
df_dev.loc[matches]['caption']

53619    this bird is mostly grey with a black bill, bl...
53699    this bird has a white crown, a long black bill...
2160     all black bird with black bill and white tippe...
1866     the bird has a very large beak almost the size...
10006    this bird has a small beak and head compared t...
Name: caption, dtype: object

In [29]:
query

'a large gray bird with a long wingspan and a long black beak.'

In [1]:
# text2image retrieval
# make an images index
d = []
a = image_dict['data'][0][clip_version]
d.append(a)

NameError: name 'image_dict' is not defined